<a href="https://colab.research.google.com/github/herysedra/covid19-mankaiza-clone/blob/andrana/stats/Fampita_mdg_ke.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
!pip install pingouin

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats
import matplotlib
import pickle

import pymc3 as pm
import theano.tensor as tt
import theano
import datetime
import time
import pingouin 

#from pingouin import corr
import pandas.util.testing as tm

cco_cases_mdg = pd.read_csv("/content/drive/My Drive/data/cco mdg.csv")

tilyvoa = pd.read_csv("/content/drive/My Drive/data/tilyvoa.csv")

Kenyans_data = pd.read_csv("/content/drive/My Drive/data/Kenyans data.csv")

mdg_databe = pd.read_csv("/content/drive/My Drive/data/databe mdg.csv")


cco_cases_mdg.set_index('Date', inplace=True)

tilyvoa.set_index('Daty', inplace=True)



path_to_save = '/content/drive/My Drive/sary/'
path_data = '/content/drive/My Drive/data/'

In [105]:
mdg_databe.info()
mdg_databe 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 216 entries, 0 to 215
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Loharano  216 non-null    object 
 1   Lnav      118 non-null    object 
 2   Taona     115 non-null    float64
 3   Daty      215 non-null    object 
 4   Toerana   47 non-null     object 
 5   Distrika  73 non-null     object 
 6   Faritra   169 non-null    object 
 7   Avaiza    44 non-null     object 
 8   Fiaviana  37 non-null     object 
 9   Ivao      192 non-null    float64
dtypes: float64(2), object(8)
memory usage: 17.0+ KB


,Loharano,Lnav,Taona,Daty,Toerana,Distrika,Faritra,Avaiza,Fiaviana,Ivao
0,FB-TVM,lahy,26.0,5/07,Tanamborozano,Toamasina t,Toamasina,NaN,NaN,1.0
1,FB-TVM,lahy,42.0,5/07,Ambohijafy Avaratra,Toamasina t,Toamasina,NaN,NaN,1.0
2,FB-TVM,vavy,26.0,5/07,Morarano,Toamasina t,Toamasina,NaN,NaN,1.0
3,FB-TVM,vavy,27.0,5/07,Morarano,Toamasina t,Toamasina,NaN,NaN,1.0
4,FB-TVM,vavy,43.0,5/07,Morafeno,Toamasina t,Toamasina,NaN,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...
211,FB-TVM,NaN,NaN,3/26,NaN,NaN,Antananarivo,NaN,NaN,NaN
212,FB-TVM,NaN,NaN,3/26,NaN,NaN,Toamasina,NaN,NaN,NaN
213,FB-TVM,NaN,NaN,3/25,NaN,NaN,NaN,NaN,NaN,0.0
214,FB-TVM,NaN,NaN,3/25,NaN,NaN,Antananarivo,NaN,NaN,NaN


In [106]:
databefar = mdg_databe.groupby('Faritra')
for Faritra in databefar:
    print(Faritra)

('Antananarivo',     Loharano  Lnav  Taona  Daty  ...       Faritra Avaiza Fiaviana Ivao
11    FB-TVM  vavy   24.0  5/07  ...  Antananarivo    NaN      NaN  1.0
12    FB-TVM  vavy   63.0  5/07  ...  Antananarivo    NaN      NaN  1.0
13    FB-TVM  lahy   25.0  5/07  ...  Antananarivo    NaN      NaN  1.0
14    FB-TVM  vavy   30.0  5/07  ...  Antananarivo    NaN      NaN  1.0
17    FB-TVM  vavy    9.0  5/07  ...  Antananarivo    NaN      NaN  1.0
..       ...   ...    ...   ...  ...           ...    ...      ...  ...
198   FB-TVM   NaN    NaN  3/29  ...  Antananarivo    NaN      NaN  NaN
200   FB-TVM   NaN    NaN  3/28  ...  Antananarivo    NaN      NaN  NaN
205   FB-TVM   NaN    NaN  3/27  ...  Antananarivo    NaN      NaN  NaN
211   FB-TVM   NaN    NaN  3/26  ...  Antananarivo    NaN      NaN  NaN
214   FB-TVM   NaN    NaN  3/25  ...  Antananarivo    NaN      NaN  NaN

[87 rows x 10 columns])
('Mahatsiatra Ambony',    Loharano  Lnav  Taona  Daty  ...             Faritra Avaiza Fiaviana

In [107]:
Antanab = databefar.get_group('Antananarivo')
Toamab = databefar.get_group('Toamasina')

Antanab
#type(Antanab)
# Antana eto dia fitambarana (key= Faritra iray,object=data momba ilay Faritra)
# Antanadaty = Antana.groupby('Daty')
# Antanadaty dia 
#plt.figure()
#plt.plot(Antana['Daty'], Antana['Ivao'], marker ="+",ls = 'None', color='black')

,Loharano,Lnav,Taona,Daty,Toerana,Distrika,Faritra,Avaiza,Fiaviana,Ivao
11,FB-TVM,vavy,24.0,5/07,Amboditsiry,Tana t,Antananarivo,NaN,NaN,1.0
12,FB-TVM,vavy,63.0,5/07,Ambohimanarina,Tana t,Antananarivo,NaN,NaN,1.0
13,FB-TVM,lahy,25.0,5/07,Anosizato,Tana t,Antananarivo,NaN,NaN,1.0
14,FB-TVM,vavy,30.0,5/07,Ambolokandrina,Tana t,Antananarivo,NaN,NaN,1.0
17,FB-TVM,vavy,9.0,5/07,Mahatony,Tana t,Antananarivo,NaN,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...
198,FB-TVM,NaN,NaN,3/29,NaN,NaN,Antananarivo,NaN,NaN,NaN
200,FB-TVM,NaN,NaN,3/28,NaN,NaN,Antananarivo,NaN,NaN,NaN
205,FB-TVM,NaN,NaN,3/27,NaN,NaN,Antananarivo,NaN,NaN,NaN
211,FB-TVM,NaN,NaN,3/26,NaN,NaN,Antananarivo,NaN,NaN,NaN


In [141]:
databefardat = mdg_databe.groupby(['Faritra', 'Daty'])
g = databefardat.apply(lambda er: er.sum(min_count=1) )
f = databefardat.sum(min_count=1)

f.loc['Antananarivo','Ivao']

# lay min_count = 1 no mikajy ny sum(nan) = nan.
# databefardat.get_group('Antananarivo') tsy mety
#g.loc['Antananarivo', 'Ivao'] tsy mety

#antana.plot(marker ="+",ls = 'None', color='black')
#dfd = np.array(antana)
#print(antana)
#print(dfd)

Daty
3/25     NaN
3/26     NaN
3/27     NaN
3/28     NaN
3/29     NaN
3/30     0.0
3/31     1.0
4/01     NaN
4/02     NaN
4/03     NaN
4/04     0.0
4/05     NaN
4/06     4.0
4/07     3.0
4/08     NaN
4/09     0.0
4/10     3.0
4/11     0.0
4/12     1.0
4/13     0.0
4/14     2.0
4/15     NaN
4/17     3.0
4/18     2.0
4/19     1.0
4/20     0.0
4/21     0.0
4/22     0.0
4/23     0.0
4/24     NaN
4/25     0.0
4/26     0.0
4/27     3.0
4/28     0.0
4/29     0.0
4/30     0.0
5/01     4.0
5/02     0.0
5/03     7.0
5/04     0.0
5/05     0.0
5/06     1.0
5/07    22.0
Name: Ivao, dtype: float64

In [95]:
daty_lav = datetime.date(2020,4,1)
formatf_date = lambda date_py: '{}/{}'.format(date_py.day, date_py.month)

daty_lst = []


for el in range(14):
  daty_lav =  daty_lav + datetime.timedelta(days = 1)
  daty = formatf_date(daty_lav)
  daty_lst.append(daty)



df = pd.DataFrame({ 'Date': daty_lst, 'Val' : 0}) 
df

,Date,Val
0,2/4,0
1,3/4,0
2,4/4,0
3,5/4,0
4,6/4,0
5,7/4,0
6,8/4,0
7,9/4,0
8,10/4,0
9,11/4,0


In [0]:
#plt.figure()
#plt.plot(Antana['Daty'], Antana['Ivao'], marker ="+",ls = 'None', color='black')

In [34]:
Kenyans_data.info()
Kenyans_data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 201 entries, 0 to 200
Data columns (total 13 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Monitored/close contacts          12 non-null     object 
 1   Discharged after being monitored  9 non-null      float64
 2   Followed up                       7 non-null      float64
 3   Daty                              201 non-null    object 
 4   Faritra                           195 non-null    object 
 5   Tilytot                           11 non-null     float64
 6   Tilyvao                           14 non-null     float64
 7   Itot                              59 non-null     float64
 8   Ivao                              159 non-null    float64
 9   Rtot                              11 non-null     float64
 10  Rvao                              18 non-null     float64
 11  Matot                             10 non-null     float64
 12  Matvao  

,Monitored/close contacts,Discharged after being monitored,Followed up,Daty,Faritra,Tilytot,Tilyvao,Itot,Ivao,Rtot,Rvao,Matot,Matvao
0,NaN,NaN,NaN,3/13,Kenya,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN
1,17,NaN,NaN,3/14,Kenya,NaN,NaN,3.0,2.0,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,3/15,Kenya,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,3/16,Kenya,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,3/17,Kenya,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,NaN,NaN,NaN,5/10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
197,NaN,NaN,NaN,5/11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
198,NaN,NaN,NaN,5/12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
199,NaN,NaN,NaN,5/13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
datacounty = Kenyans_data.groupby('Faritra')
for Faritra in datacounty:
    print(Faritra)
Nairodata = datacounty.get_group('Nairobi')
Mombadata = datacounty.get_group('Mombasa')


('Bungoma',     Monitored/close contacts  Discharged after being monitored  ...  Matot Matvao
172                      NaN                               NaN  ...    NaN    NaN

[1 rows x 13 columns])
('Buruburu',    Monitored/close contacts  Discharged after being monitored  ...  Matot Matvao
92                      NaN                               NaN  ...    NaN    NaN

[1 rows x 13 columns])
('Casarani',     Monitored/close contacts  Discharged after being monitored  ...  Matot Matvao
140                      NaN                               NaN  ...    NaN    NaN

[1 rows x 13 columns])
('City Park',     Monitored/close contacts  Discharged after being monitored  ...  Matot Matvao
131                      NaN                               NaN  ...    NaN    NaN

[1 rows x 13 columns])
('Dandora',     Monitored/close contacts  Discharged after being monitored  ...  Matot Matvao
132                      NaN                               NaN  ...    NaN    NaN
142                   

In [0]:
Nairodata 